In [1]:

import torch
from numba import cuda
cuda.select_device(0)
cuda.close()
cuda.select_device(0)

<weakproxy at 0x7f9c86bcaa90 to Device at 0x7f9c86bcbd90>

In [2]:
import pandas as pd
import numpy as np
import os

data_dir = "/home/peterr/macocu/task8/transfer_10/"
train = pd.read_csv("17_train.csv" )
train["split"] = "train"

dev = pd.read_csv("17_dev.csv" )
dev["split"] = "dev"


df = pd.concat([train, dev], ignore_index=True).reset_index(drop=True)
df["path"] = data_dir + df["hashname"]
df["sentence"] = df.unnormalized_transcript
df = df.loc[:, ["path", "sentence", "split"]]

modelname = "19_unnormalised"

# Use old or new vocab?
os.system("cp vocab_300_with_numbers.json vocab.json")

0

In [3]:
common_voice_train_df = df.loc[df.split=="train", :]
common_voice_test_df = df.loc[df.split == "dev", :]

common_voice_train_df.reset_index(drop=True, inplace=True)
common_voice_test_df.reset_index(drop=True, inplace=True)

In [4]:
common_voice_train_df["audio"] = common_voice_train_df.path
common_voice_test_df["audio"] = common_voice_test_df.path
import datasets
from datasets import load_dataset, load_metric, Audio

common_voice_train_dataset = datasets.Dataset.from_pandas(common_voice_train_df)
common_voice_test_dataset = datasets.Dataset.from_pandas(common_voice_test_df)

common_voice_train_dataset = common_voice_train_dataset.cast_column("audio", Audio())
common_voice_test_dataset = common_voice_test_dataset.cast_column("audio", Audio())

<ipython-input-4-1ea4e411157d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  common_voice_train_df["audio"] = common_voice_train_df.path
<ipython-input-4-1ea4e411157d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  common_voice_test_df["audio"] = common_voice_test_df.path


In [5]:


from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(
    "./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token=" ")

feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

processor = Wav2Vec2Processor(
    feature_extractor=feature_extractor, tokenizer=tokenizer)

import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)


from transformers import Trainer
from transformers import TrainingArguments
from transformers import Wav2Vec2ForCTC
from typing import Any, Dict, List, Optional, Union
from dataclasses import dataclass, field
import torch


def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(
        audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch


common_voice_train_mapped = common_voice_train_dataset.map(
    prepare_dataset, remove_columns=common_voice_train_dataset.column_names)
common_voice_test_mapped = common_voice_test_dataset.map(
    prepare_dataset, remove_columns=common_voice_test_dataset.column_names)
print("Data Preparation Complete!")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  0%|          | 0/24500 [00:00<?, ?ex/s]

  0%|          | 0/500 [00:00<?, ?ex/s]

Data Preparation Complete!


In [6]:

repo_name = modelname
wer_metric = load_metric("wer")
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

max_input_length_in_sec = 20
common_voice_train_mapped = common_voice_train_mapped.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m",
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

model.freeze_feature_extractor()

from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=4,
  evaluation_strategy="steps",
  num_train_epochs=8,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=757,
  eval_steps=757//2,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=5,
  push_to_hub=False,
)

from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train_mapped,
    eval_dataset=common_voice_test_mapped,
    tokenizer=processor.feature_extractor,
)

trainer.train()

  0%|          | 0/25 [00:00<?, ?ba/s]

Some weights of the model checkpoint at facebook/wav2vec2-xls-r-300m were not used when initializing Wav2Vec2ForCTC: ['quantizer.weight_proj.bias', 'project_hid.bias', 'quantizer.codevectors', 'project_q.bias', 'quantizer.weight_proj.weight', 'project_q.weight', 'project_hid.weight']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it 

Step,Training Loss,Validation Loss


# Looking at output transcriptions:

In [ ]:
# %%
# 
from itertools import groupby
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor, Wav2Vec2CTCTokenizer
import soundfile as sf

##############
# load model & audio and run audio through model
##############
import os
# Use old or new vocab?
# os.system("cp vocab_december.json vocab.json")
# os.system("cp vocab_300.json vocab.json")

# tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(
#     "/home/peterr/macocu/task8/", unk_token="[UNK]", 
#     pad_token="[PAD]", word_delimiter_token=" ")

# feature_extractor = Wav2Vec2FeatureExtractor(
#     feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

# processor = Wav2Vec2Processor(
#     feature_extractor=feature_extractor, tokenizer=tokenizer)



#model_name = '/home/peterr/macocu/task8/300_/checkpoint-2800'
# model_name = '/home/peterr/macocu/task8/200_/checkpoint-6000'
model_name = '/home/peterr/macocu/task8/18_normalised/checkpoint-3028'

model = Wav2Vec2ForCTC.from_pretrained(model_name).cuda()

def get_transcript(audio_filepath:str):
    speech, sample_rate = sf.read(audio_filepath)
    input_values = processor(speech, sampling_rate=sample_rate, return_tensors="pt").input_values.cuda()

    logits = model(input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.decode(predicted_ids[0]).lower()

    ##############
    # this is where the logic starts to get the start and end timestamp for each word
    ##############
    words = [w for w in transcription.split(' ') if len(w) > 0]
    predicted_ids = predicted_ids[0].tolist()
    duration_sec = input_values.shape[1] / sample_rate


    ids_w_time = [(i / len(predicted_ids) * duration_sec, _id) for i, _id in enumerate(predicted_ids)]
    # remove entries which are just "padding" (i.e. no characers are recognized)
    ids_w_time = [i for i in ids_w_time if i[1] != processor.tokenizer.pad_token_id]
    # now split the ids into groups of ids where each group represents a word
    split_ids_w_time = [list(group) for k, group
                        in groupby(ids_w_time, lambda x: x[1] == processor.tokenizer.word_delimiter_token_id)
                        if not k]
    assert len(split_ids_w_time) == len(words)  # make sure that there are the same number of id-groups as words. Otherwise something is wrong

    word_start_times = []
    word_end_times = []
    for cur_ids_w_time, cur_word in zip(split_ids_w_time, words):
        _times = [_time for _time, _id in cur_ids_w_time]
        word_start_times.append(min(_times))
        word_end_times.append(max(_times))

    return words, word_start_times, word_end_times



In [ ]:
" ".join(get_transcript("/home/peterr/macocu/task8/transfer_10/sPNuDebATRM_1853.82-1873.66.wav")[0])

In [ ]:
dev = pd.read_csv("17_dev.csv")
test= pd.read_csv("17_test.csv")
dev.head()

In [ ]:
dev[modelname+"_output"] = dev.hashname.apply(
    lambda s: " ".join(get_transcript(f"{data_dir}{s}")[0])
)

In [ ]:
test= pd.read_csv("17_test.csv")
test[modelname+"_output"] = test.hashname.apply(
    lambda s: " ".join(get_transcript(f"{data_dir}{s}")[0])
)

In [ ]:
test[["human_transcript", "18_normalised_output"]].sample(10)

In [ ]:
dev.to_csv("17_dev.csv", index=False)
test.to_csv("17_test.csv", index=False)